In [1]:
import os
import yaml
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain_community.utilities.requests import RequestsWrapper
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def load_yaml_spec(file_path:str):
    # Load Swagger Specification
    with open(file_path, "r", encoding='utf8') as f:
        raw_waapi_api_spec = yaml.load(f, Loader=yaml.Loader)

    # Add the host from environment variable
    waapi_host = os.getenv("WAAPI_HOST")
    if waapi_host:
        raw_waapi_api_spec["servers"] = [{"url": waapi_host}]

    # Reduce the OpenAPI spec
    return reduce_openapi_spec(raw_waapi_api_spec)


In [2]:
api_spec=load_yaml_spec("waapi.yaml")

In [15]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

def count_tokens(s):
    return len(enc.encode(s))

count_tokens(yaml.dump(api_spec))

134615

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits.openapi.planner_prompt import API_PLANNER_PROMPT

endpoint_descriptions = [
    f"{name} {description}" for name, description, _ in api_spec.endpoints
]
prompt = PromptTemplate(
    template=API_PLANNER_PROMPT,
    input_variables=["query"],
    partial_variables={"endpoints": "- " + "- ".join(endpoint_descriptions)},
)

In [20]:
prompt.partial_variables

{'endpoints': '- GET /api/{session}/auth/qr None- POST /api/{session}/auth/request-code None- POST /api/{session}/auth/authorize-code None- GET /api/{session}/auth/captcha None- POST /api/{session}/auth/captcha None- GET /api/sessions None- POST /api/sessions Create session a new session (and start it at the same time if required). The requestBody has a placeholder for session name which should be replaced by user\'s phone number- GET /api/sessions/{session} None- PUT /api/sessions/{session} - DELETE /api/sessions/{session} Delete the session with the given name. Stop and logout as well. Idempotent operation.- GET /api/sessions/{session}/me None- POST /api/sessions/{session}/start Start the session with the given name. The session must exist. Idempotent operation.- POST /api/sessions/{session}/stop Stop the session with the given name. Idempotent operation.- POST /api/sessions/{session}/logout Logout the session, restart a session if it was not STOPPED- POST /api/sessions/{session}/res

In [18]:
#prompt.partial_variables

count_tokens(str(prompt))

2334

In [8]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0, api_key=OPENAI_API_KEY)
api_planner_tool = planner._create_api_planner_tool(api_spec, llm)

# Set headers for the requests
headers = {"X-API-KEY": os.getenv("WAAPI_API_KEY")}
requests_wrapper = RequestsWrapper(headers=headers)

api_controller_tool=planner._create_api_controller_tool(
            api_spec,
            requests_wrapper,
            llm,
            allow_dangerous_requests=True,
            allowed_operations=("GET", "POST")
        )

In [5]:
llm_with_tools = llm.bind_tools([api_planner_tool, api_controller_tool])
response=llm_with_tools.invoke("I want to sign up for this service. What APIs should I call?")


In [9]:
print(response.tool_calls)

[{'name': 'api_planner', 'args': {'__arg1': 'sign_up'}, 'id': 'call_PQxK7UB3LZxhQkfqBiZVClfi', 'type': 'tool_call'}]


In [10]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([api_planner_tool, api_controller_tool])
tool_node.invoke({"messages": [llm_with_tools.invoke("I want to sign up for this service. What APIs should I call?")]})

{'messages': [ToolMessage(content='1. POST /api/sessions to create a new session for the user.\n2. GET /api/{session}/auth/qr to generate a QR code for the user to scan and authenticate their account.', name='api_planner', tool_call_id='call_2br4slkUCJPRCQnfE8Phpo8M')]}

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()
#I want to sign up for this service. Which APIs should I call?
user_input = input("Enter your message: ")
while user_input:
    for chunk in app.stream(
        {"messages": [("human", user_input)]}, stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()
        user_input = input("Enter your message: ")
        


================================ Human Message =================================

I want to sign up for this service. Which APIs should I call?
================================== Ai Message ==================================
Tool Calls:
  api_planner (call_QvBUEKeRk4cdMPkN7vfxZ4Ei)
 Call ID: call_QvBUEKeRk4cdMPkN7vfxZ4Ei
  Args:
    __arg1: sign up
================================= Tool Message =================================
Name: api_planner

1. POST /api/sessions to create a new session for the user.
2. GET /api/{session}/auth/qr to generate a QR code for the user to scan and authenticate their account.
================================== Ai Message ==================================

To sign up for this service, you should call the following APIs:

1. `POST /api/sessions` to create a new session for the user.
2. `GET /api/{session}/auth/qr` to generate a QR code for the user to scan and authenticate their account.
================================ Human Message ====================

KeyboardInterrupt: 